In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import polars as pl
from pathlib import Path
import data_visualization as dv
from time import time
import copy
import importlib
import GTO
import dyad_gt_params as param
import constants as const
import plot_functions as pf
import model_functions as mf


# Notebook Outline

# Parameter Setup

In [4]:
HIT_TIME = 0.8
h=0.01

Q1 = mf.generate_Q(
    weight_dict=param.Q1_WEIGHTS, 
    state_mapping=param.state_mapping, 
    cross_terms=[["ccx","ctx"], ["ccy","cty"], 
                 ["rhy","lhy"]], 
    QVAL=param.QVAL
)
Q2 = mf.generate_Q(
    weight_dict=param.Q2_WEIGHTS, 
    state_mapping=param.state_mapping, 
    cross_terms=[["ccx","ctx"], ["ccy","cty"],
                    ["rhy","lhy"]], 
    QVAL = param.QVAL,
)

Q1_xvals = [
    param.IRREL_QVAL #  Both irrelevant
    param.QVAL, # P1 Relevant
    param.IRREL_QVAL, # P2 Relevant
    param.QVAL, # Both Relevant
]
Q2_xvals = [
    param.IRREL_QVAL #  Both irrelevant
    param.IRREL_QVAL, # P1 Relevant
    param.QVAL, # P2 Relevant
    param.QVAL, # Both Relevant
]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1765650026.py, line 20)